In [2]:
import numpy as np
import torch

from torch import nn

In [3]:
from video import Video

video_path = "/h/mhasan/ece496-capstone/train/sample/video.mp4"
transcript_path = "/h/mhasan/ece496-capstone/train/sample/transcript.vtt"
transcript2_path = "/h/mhasan/ece496-capstone/train/sample/transcript2.vtt"

v = Video(video_path, transcript_path)
v.align()
v.generate_frames("sample", swap=True)

v2 = Video(video_path, transcript2_path)
v2.align()
v2.generate_frames("sample", swap=True)

In [4]:
import glob
import itertools

from loss import loss_RA_MIL
from detector import Detector
from parser import parse

detector = Detector()

loading configuration file cache
loading weights file https://cdn.huggingface.co/unc-nlp/frcnn-vg-finetuned/pytorch_model.bin from cache at /h/mhasan/.cache/torch/transformers/57f6df6abe353be2773f2700159c65615babf39ab5b48114d2b49267672ae10f.77b59256a4cf8343ae0f923246a81489fc8d82f98d082edc2d2037c977c0d9d0
All model checkpoint weights were used when initializing GeneralizedRCNN.

All the weights of GeneralizedRCNN were initialized from the model checkpoint at unc-nlp/frcnn-vg-finetuned.
If your task is similar to the task the model of the checkpoint was trained on, you can already use GeneralizedRCNN for predictions without further training.


In [5]:
import importlib
import loss
importlib.reload(loss)

<module 'loss' from '/scratch/ssd002/home/mhasan/ece496-capstone/train/loss.py'>

In [93]:
loss.loss_RA_MIL(0.5, torch.randn(2,5,5), torch.randn(2,5,3,100), torch.randn(2,5,3,100))

tensor(296.7588)

In [6]:
images = sorted(glob.glob("/h/mhasan/ece496-capstone/train/sample/*.png"))
candidates = [detector.inference(image, max_detections=5) for image in images]

In [7]:
NULL = '[unused1]'
PAD = '[unused2]'
ENTITY = '[unused3]'
ACTION = '[SEP]'

MAX_STEP_LENGTH = 30

In [8]:
steps1 = [step.text.strip() for step in v.steps]
steps2 = [step.text.strip() for step in v2.steps]

In [9]:
from transformers import LxmertModel, LxmertTokenizer

tokenizer = LxmertTokenizer.from_pretrained("unc-nlp/lxmert-base-uncased", pad_token=PAD)
tokenizer.add_special_tokens({"additional_special_tokens": [NULL, PAD, ENTITY]})
tokenizer.encode([NULL, PAD, ENTITY], add_special_tokens=True)

tokens_steps1 = tokenizer(
                    steps1,
                    return_token_type_ids=False,
                    return_attention_mask=False,
                    add_special_tokens=True,
                    padding="max_length",
                    max_length=MAX_STEP_LENGTH + 2,
                    return_tensors="pt"
                )

tokens_steps2 = tokenizer(
                    steps2,
                    return_token_type_ids=False,
                    return_attention_mask=False,
                    add_special_tokens=True,
                    padding="max_length",
                    max_length=MAX_STEP_LENGTH + 2,
                    return_tensors="pt"
                )

PyTorch version 1.6.0 available.


In [10]:
steps1_flat = tokens_steps1['input_ids'].flatten()
steps1_flat = steps1_flat[steps1_flat != 101]
steps1_flat = tokenizer.decode(steps1_flat) + ' ' + NULL

In [11]:
steps2_flat = tokens_steps2['input_ids'].flatten()
steps2_flat = steps2_flat[steps2_flat != 101]
steps2_flat = tokenizer.decode(steps2_flat) + ' ' + NULL

In [12]:
# Find all of the bounding boxes for the detections and their features.
boxes = torch.tensor([candidate[0].numpy() for candidate in candidates]).squeeze(1)
features = torch.tensor([candidate[1].numpy() for candidate in candidates]).squeeze(1)

boxes = boxes.flatten(start_dim=0, end_dim=1)
features = features.flatten(start_dim=0, end_dim=1)

boxes = torch.stack((boxes, boxes))
features = torch.stack((features, features))

In [11]:
model = Model()

NameError: name 'Model' is not defined

In [13]:
steps = [steps1_flat, steps2_flat]

ENTITIES_COUNT = [
    [2, 2, 1, 2, 3, 2, 2, 2, 1, 1],
    [1, 1, 1, 1, 3, 2, 1, 2, 1, 1]
]

In [14]:
steps1

['Grill [unused3] the tomatoes in [unused3] a pan.',
 'Add [unused3] oil into [unused3] the pan.',
 'Cook [unused3] the bacon.',
 'Spread [unused3] some mayonnaise onto [unused3] the bread.',
 'Place [unused3] a piece of [unused3] lettuce onto [unused3] it.',
 'Place [unused3] the tomatoes over [unused3] it.',
 'Sprinkle [unused3] some salt and pepper onto [unused3] it.',
 'Place [unused3] the bacon at [unused3] the top.',
 'Place the [unused3] piece of bread at the top.']

In [15]:
steps2

['Grill [unused3] the tomatoes.',
 'Add [unused3] oil.',
 'Cook [unused3] the bacon.',
 'Spread [unused3] some mayonnaise.',
 'Place [unused3] a piece of [unused3] lettuce onto [unused3] it.',
 'Place [unused3] the tomatoes over [unused3] it.',
 'Sprinkle some pepper onto [unused3] it.',
 'Place [unused3] the bacon at [unused3] the top.',
 'Place the [unused3] piece of bread at the top.']

In [16]:
# https://github.com/google-research/bert/issues/635
# https://colab.research.google.com/drive/18TyuMfZYlgQ_nXo-tr8LCnzUaoX0KS-h?usp=sharing#scrollTo=W4cZIVrg82ua.

import itertools
import torch
import einops

from transformers import LxmertModel, LxmertTokenizer
from torch import nn
from torch.nn.utils.rnn import pad_sequence

class Model(nn.Module):
    NULL = '[unused1]'
    PAD = '[unused2]'
    ENTITY = '[unused3]'
    ACTION = '[SEP]'
    
    DETECTION_EMBEDDING_SIZE = 2048
    OUTPUT_EMBEDDING_SIZE = 768
        
    def __init__(self):
        super(Model, self).__init__()
        
        self.lxmert_tokenizer = LxmertTokenizer.from_pretrained("unc-nlp/lxmert-base-uncased")
        self.lxmert_tokenizer.add_special_tokens({"additional_special_tokens": [NULL, PAD, ENTITY]})
        self.lxmert_tokenizer.encode([NULL, PAD, ENTITY])
        
        self.NULL_TOKEN = self.lxmert_tokenizer.convert_tokens_to_ids(NULL)
        self.ENTITY_TOKEN = self.lxmert_tokenizer.convert_tokens_to_ids(ENTITY)
        self.ACTION_TOKEN = self.lxmert_tokenizer.convert_tokens_to_ids(ACTION)
        
        self.lxmert = LxmertModel.from_pretrained("unc-nlp/lxmert-base-uncased")
        
    def forward(self, MAX_INSTRUCTION_LENGTH, BATCH_SIZE, NUM_ACTIONS, CANDIDATES, steps, features, boxes, entity_count):
        '''
            MAX_INSTRUCTION_LENGTH
                : maximum number of words in a combined string of steps for a video,
                  this instruction string must contain the NULL step.
                  
                  ex. | E ... E . [SEP] [PAD] [PAD] | .... [SEP] [PAD] | NULL | 
                  
            steps
                : batched video steps of size (BATCH_SIZE, NUM_ACTIONS), list of lists,
                  each instructional video must have the same number of steps = NUM_ACTIONS.
                
                  ex. [['step 1.1', 'step 1.2'], ['step 2.1', 'step 2.2']]
                
            features
                : batched detection features of size (BATCH_SIZE, CANDIDATES * NUM_ACTIONS, 2048)
                
            boxes
                : batched bounding boxes of detection features of size (BATCH_SIZE, CANDIDATES * NUM_ACTIONS, 4)
                
            entity_count:
                : number of entities per action, size (BATCH_SIZE, NUM_ACTIONS), list of lists
        '''

        inputs = self.lxmert_tokenizer(
            steps,
            padding="max_length",
            max_length= MAX_INSTRUCTION_LENGTH + 2, # [CLS] and [SEP] token
            truncation=False,
            return_token_type_ids=True,
            return_attention_mask=True,
            add_special_tokens=True,
            return_tensors="pt"
        )

        output = self.lxmert(
            input_ids=inputs.input_ids,
            attention_mask=inputs.attention_mask,
            visual_feats=features,
            visual_pos=boxes,
            token_type_ids=inputs.token_type_ids,
            return_dict=True,
            output_attentions=True
        )
        
        token_ids = inputs.input_ids
        
        entity_idx = ((token_ids == self.ENTITY_TOKEN) | (token_ids == self.NULL_TOKEN))
        action_idx = (token_ids == self.ACTION_TOKEN)
        
        entity_embeddings = output['language_output'][entity_idx]
        action_embeddings = output['language_output'][action_idx]
        vision_embeddings = output['vision_output']
        
        split_sizes = torch.tensor(entity_count).flatten().tolist()
        entities = entity_embeddings.split(split_sizes)
        
        E = pad_sequence(entities, batch_first=True)
        max_entities = E.shape[1]
        E = E.reshape(-1, NUM_ACTIONS, E.shape[1], E.shape[2])

        A = action_embeddings.reshape(BATCH_SIZE, NUM_ACTIONS, -1)
        V = vision_embeddings.reshape(BATCH_SIZE, NUM_ACTIONS, CANDIDATES, -1)
        
        # Calculate loss_E.
        loss_E = E
        
        # Calculate VG (VG_scores_index) and loss_V.
        VG_scores = torch.einsum('bacs, baes -> baec', V, E)
        VG_scores_max, VG_scores_index = VG_scores.max(dim=-1)

        V_flat = V.reshape(-1, self.OUTPUT_EMBEDDING_SIZE)

        VG_scores_index_flat = VG_scores_index.flatten()
        offsets = torch.arange(0, BATCH_SIZE * NUM_ACTIONS * CANDIDATES, CANDIDATES)
        offsets = offsets.repeat_interleave(max_entities)

        VG_scores_index_flat = VG_scores_index_flat + offsets

        loss_V = V_flat[VG_scores_index_flat, :].reshape(BATCH_SIZE, NUM_ACTIONS, max_entities, -1)
        
        # Calculate RR (RR_scores_index).
        #note: we write c here as number of actions as well
        #since we don't wish to sum over a 
        RR_scores = torch.einsum('baes, bcs -> baec', E, A)

        edge_mask = torch.ones(NUM_ACTIONS, NUM_ACTIONS).tril(diagonal=-1)
        edge_mask[-1, :] = 0
        edge_mask[:, -1] = 1
        edge_mask = einops.repeat(edge_mask, 'x y -> b x c y', b=BATCH_SIZE, c=max_entities)

        RR_scores_max, RR_scores_index = (RR_scores * edge_mask).max(dim=-1)
        
        # Calculate loss_R.
        loss_R = torch.ones((BATCH_SIZE, NUM_ACTIONS, NUM_ACTIONS))

        dim_1 = RR_scores_index.reshape(BATCH_SIZE, -1)
        dim_2 = torch.arange(NUM_ACTIONS).repeat_interleave(max_entities)
        dim_2 = einops.repeat(dim_2, 'd -> b d', b=BATCH_SIZE)

        loss_R[:, dim_1, dim_2] = 0.0
        
        #entity embeddings, selected visual grounding embeddings, adjacency list for
        #ref resolution
        return loss_E, loss_V, loss_R

In [17]:
def train(model, epochs=25, lr=0.001, batch_size=10, y=0.5):
    '''
    Training loop for the model.
    '''
    
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    
    # Ouput losses.
    train_loss = np.zeros(epochs)
    
    # Output accuracies.
    train_accuracy = np.zeros(epochs)
    
    m_RR = None
    m_VG = None
    
    model.train()
    
    for epoch in range(epochs):
        # Zero out any gradients.
        optimizer.zero_grad()

        # Run inference (forward pass).
        loss_E, loss_V, loss_R = model(265, 2, 10, 5, steps, features, boxes, ENTITIES_COUNT)

        # Loss from alignment.
        loss_ = loss.loss_RA_MIL(y, loss_R, loss_E, loss_V)
        print(loss_)

        # Backpropagation (backward pass).
        loss_.backward()

        # Update parameters.
        optimizer.step()

        #m_RR = RR
        #m_VG = VG
            
        # TODO: save loss and accuracy at each epoch, plot (and checkpoint).
        train_loss[epoch] = loss_
        
    return train_loss

In [ ]:
model = Model()
train_loss = train(model, epochs=50, lr=0.0001)

tensor(397.5083, grad_fn=<SumBackward0>)
tensor(3777.6223, grad_fn=<SumBackward0>)
tensor(1839.2795, grad_fn=<SumBackward0>)
tensor(1486.8861, grad_fn=<SumBackward0>)
tensor(1778.6123, grad_fn=<SumBackward0>)
tensor(1102.3772, grad_fn=<SumBackward0>)
tensor(1398.0608, grad_fn=<SumBackward0>)


In [ ]:
import matplotlib.pyplot as plt
plt.plot(train_loss)

In [25]:
model

Model(
  (lxmert): LxmertModel(
    (embeddings): LxmertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768, padding_idx=0)
      (token_type_embeddings): Embedding(2, 768, padding_idx=0)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): LxmertEncoder(
      (visn_fc): LxmertVisualFeatureEncoder(
        (visn_fc): Linear(in_features=2048, out_features=768, bias=True)
        (visn_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (box_fc): Linear(in_features=4, out_features=768, bias=True)
        (box_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (layer): ModuleList(
        (0): LxmertLayer(
          (attention): LxmertSelfAttentionLayer(
            (self): LxmertAttention(
              (query): Linear(in_features=

In [33]:
boxes.size()

torch.Size([2, 50, 4])

In [26]:
steps[0]

'grill [unused3] the tomatoes in [unused3] a pan. [SEP] [unused2] [unused2] [unused2] [unused2] [unused2] [unused2] [unused2] [unused2] [unused2] [unused2] [unused2] [unused2] [unused2] [unused2] [unused2] [unused2] [unused2] [unused2] [unused2] [unused2] [unused2] add [unused3] oil into [unused3] the pan. [SEP] [unused2] [unused2] [unused2] [unused2] [unused2] [unused2] [unused2] [unused2] [unused2] [unused2] [unused2] [unused2] [unused2] [unused2] [unused2] [unused2] [unused2] [unused2] [unused2] [unused2] [unused2] [unused2] cook [unused3] the bacon. [SEP] [unused2] [unused2] [unused2] [unused2] [unused2] [unused2] [unused2] [unused2] [unused2] [unused2] [unused2] [unused2] [unused2] [unused2] [unused2] [unused2] [unused2] [unused2] [unused2] [unused2] [unused2] [unused2] [unused2] [unused2] [unused2] spread [unused3] some mayonnaise onto [unused3] the bread. [SEP] [unused2] [unused2] [unused2] [unused2] [unused2] [unused2] [unused2] [unused2] [unused2] [unused2] [unused2] [unused2]

In [46]:
model = Model()
loss_E, loss_V, loss_R = model(265, 2, 10, 5, steps, features, boxes, ENTITIES_COUNT)

In [98]:
torch.tensor(loss_R)

tensor([[[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
         [0., 1., 1., 1., 1., 1., 1., 1., 1., 0.]],

        [[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
         [0., 1., 1., 1., 1.,

In [58]:
features.size()

torch.Size([2, 50, 2048])

In [ ]:
 MAX_INSTRUCTION_LENGTH, BATCH_SIZE, NUM_ACTIONS, CANDIDATES, steps, features, boxes, entity_count):
        '''
            MAX_INSTRUCTION_LENGTH
                : maximum number of words in a combined string of steps for a video,
                  this instruction string must contain the NULL step.
                  
                  ex. | E ... E . [SEP] [PAD] [PAD] | .... [SEP] [PAD] | NULL | 
                  
            steps
                : batched video steps of size (BATCH_SIZE, NUM_ACTIONS), list of lists,
                  each instructional video must have the same number of steps = NUM_ACTIONS.
                
                  ex. [['step 1.1', 'step 1.2'], ['step 2.1', 'step 2.2']]
                
            features
                : batched detection features of size (BATCH_SIZE, CANDIDATES * NUM_ACTIONS, 2048)
                
            boxes
                : batched bounding boxes of detection features of size (BATCH_SIZE, CANDIDATES * NUM_ACTIONS, 4)
                
            entity_count:
                : number of entities per action, size (BATCH_SIZE, NUM_ACTIONS), list of lists
        '''

In [19]:
        inputs = tokenizer(
            steps,
            padding="max_length",
            max_length= 265 + 2, # [CLS] and [SEP] token
            truncation=False,
            return_token_type_ids=True,
            return_attention_mask=True,
            add_special_tokens=True,
            return_tensors="pt"
        )

In [18]:
len(steps[1].split(' '))

265

In [27]:
inputs['input_ids']

tensor([[  101, 18651,     4,  1996, 12851,  1999,     4,  1037,  6090,  1012,
           102,     3,     3,     3,     3,     3,     3,     3,     3,     3,
             3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
             3,     3,  5587,     4,  3514,  2046,     4,  1996,  6090,  1012,
           102,     3,     3,     3,     3,     3,     3,     3,     3,     3,
             3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
             3,     3,     3,  5660,     4,  1996, 11611,  1012,   102,     3,
             3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
             3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
             3,     3,     3,     3,  3659,     4,  2070, 14415,  9516,  5562,
          3031,     4,  1996,  7852,  1012,   102,     3,     3,     3,     3,
             3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
             3,     3,     3,     3,     3,  2173,  

In [30]:
decoded_steps = tokenizer.batch_decode(inputs['input_ids'])

In [32]:
len(decoded_steps[0].split(' '))

267

In [21]:
lxmert = LxmertModel.from_pretrained("unc-nlp/lxmert-base-uncased")

In [22]:
        output = lxmert(
            input_ids=inputs.input_ids,
            attention_mask=inputs.attention_mask,
            visual_feats=features,
            visual_pos=boxes,
            token_type_ids=inputs.token_type_ids,
            return_dict=True,
            output_attentions=True
        )

In [25]:
output['language_output'].size()

torch.Size([2, 282, 768])

In [39]:
NUM_ACTIONS=10
BATCH_SIZE=2

edge_mask = torch.ones(NUM_ACTIONS, NUM_ACTIONS).tril(diagonal=-1)
edge_mask[-1, :] = 0
edge_mask[:, -1] = 1
edge_mask = einops.repeat(edge_mask, 'x y -> b x c y', b=BATCH_SIZE, c=5)

In [41]:
edge_mask[0]

tensor([[[0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.]],

        [[1., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
         [1., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
         [1., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
         [1., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
         [1., 0., 0., 0., 0., 0., 0., 0., 0., 1.]],

        [[1., 1., 0., 0., 0., 0., 0., 0., 0., 1.],
         [1., 1., 0., 0., 0., 0., 0., 0., 0., 1.],
         [1., 1., 0., 0., 0., 0., 0., 0., 0., 1.],
         [1., 1., 0., 0., 0., 0., 0., 0., 0., 1.],
         [1., 1., 0., 0., 0., 0., 0., 0., 0., 1.]],

        [[1., 1., 1., 0., 0., 0., 0., 0., 0., 1.],
         [1., 1., 1., 0., 0., 0., 0., 0., 0., 1.],
         [1., 1., 1., 0., 0., 0., 0., 0., 0., 1.],
         [1., 1., 1., 0., 0., 0., 0., 0., 0., 1.],
         [1., 1., 1., 0.,

In [53]:
a = torch.rand(2,10, 10)

In [54]:
a.diagonal(dim1 = 1, dim2= 2).size()

torch.Size([2, 10])